In [1]:
from TwitterAPI import TwitterAPI
import numpy as np
import pandas as pd
from datetime import datetime,timedelta
from pytz import timezone
import json
import matplotlib.pyplot as plt
import time
import sys
from sys import getsizeof

# Functions

In [2]:
def getPasswords():
    file = open('passwords.txt','r')
    passwords_json = file.read()
    return json.loads(passwords_json)

def get_twitter():
    """ 
    Generates twitter connection
    """
    passwords = getPasswords()
    
    consumer_key = passwords['consumer_key']
    consumer_secret = passwords['consumer_secret']
    access_token = passwords['access_token']
    access_token_secret = passwords['access_token_secret']
    
    #Set Tokens
    '''
    consumer_key='nDx2bQS2DVdXEZd0b8MwaEG2L'
    consumer_secret='BeA1VuBq7hyi01fo4LFD13YAcrDgeYxIJTQgzx4gg8H5fSc3FR'
    access_token='900568210424164352-pKLEmPfNcqyGi3q67pyK3bJYtc5G01S'
    access_token_secret='uGbegX9whUY0FjZS6tgIrDITVOM8Q6tKMODNVPG2sT9Aq'
    '''
    
    twitter = TwitterAPI(
                   consumer_key,
                   consumer_secret,
                   access_token,
                   access_token_secret)
    return twitter

def get_NumberofFollowers(screen_name):       #does not crash when Exception, but does not correct exceptions either.
    '''
    returns the number of friends the user has
    '''
    try:
        request=twitter.request('users/lookup',{'screen_name':screen_name})
        return request.json()[0]['followers_count']
    except Exception as e:
        print (screen_name +str(request.json()['errors']))
        return np.NaN    

def get_timeline(screen_name,count=200): #should modify this to be more robust
    '''
    retuns a list with the last (count-deleted) tweets from the user
    '''
    try:
        request = twitter.request('statuses/user_timeline',{'screen_name': screen_name,'count': count})
        return request.json()
    except Exception as e:
        print (screen_name +str(request.json()['errors']))
        return np.NaN
def get_TodaysTweets(screen_name,count=200,exclude_replies = True, max_id = None): #should modify this to be more robust
    '''
    retuns a list with the last (count-deleted) tweets from the user
    '''
    try:
        request = twitter.request('statuses/user_timeline',{'screen_name': screen_name,'count': count,
                                                            'exclude_replies':exclude_replies,'max_id':max_id})
        assert (type(request.json())==list) 
    except AssertionError:
        #print(request.json())
        pb.increase()
        return np.NaN
    else:
        tweet_list = request.json()
        yesterday_DT = datetime.now(timezone('US/Central'))-timedelta(days=1)
        todaysTweets = []
        if tweet_list == []:
            pb.increase()
            return todaysTweets
        elif not (strToChicago_DT(tweet_list[-1]['created_at']) > yesterday_DT): #last of the 200 was in the last 24h
            todaysTweets =  [t for t in tweet_list if strToChicago_DT(t['created_at']) > yesterday_DT]
            pb.increase()
            return todaysTweets
        else:
            todaysTweets = tweet_list.copy()
            max_id = todaysTweets[-1]['id']
            todaysTweets += get_TodaysTweets(screen_name,count=200,exclude_replies = True, max_id = max_id)[1:]
            return todaysTweets

## Helper functions

In [3]:
def strToChicago_DT(string):
    UTC_DT = datetime.strptime(string,'%a %b %d %H:%M:%S +0000 %Y').replace(tzinfo = timezone('UTC'))
    Chicago_DT = UTC_DT.astimezone(timezone('US/Central'))
    return Chicago_DT

In [4]:
class progress_bar(object):
    def __init__(self,length,startingValue = 0):
        self.length = length
        self.progress = startingValue
    def increase(self,by = 1):
        self.progress += 1/self.length
        self.update_progress(self.progress)
    def update_progress(self,progress):
        barLength = 10 # Modify this to change the length of the progress bar
        status = ""
        if isinstance(progress, int):
            progress = float(progress)
        if not isinstance(progress, float):
            progress = 0
            status = "error: progress var must be float\r\n"
        if progress < 0:
            progress = 0
            status = "Halt...\r\n"
        if progress >= 1:
            progress = 1
            status = "Done...\r\n"
        block = int(round(barLength*progress))
        text = "\rPercent: [{0}] {1}% {2}".format( "#"*block + "-"*(barLength-block), round(progress*100,2), status)
        sys.stdout.write(text)
        sys.stdout.flush()

# Import Accounts

In [5]:
accounts_df = pd.read_csv('Data/ModelData/accounts.csv',sep=";")

# Collect Data

In [6]:
twitter = get_twitter()

In [7]:
accounts = accounts_df['Account'] #pd series
todaysTweets = pd.DataFrame(accounts, columns=['Account'])
todaysFollowers = pd.DataFrame(accounts, columns=['Account'])
date=str(datetime.now().date())
todaysTweets[date] = np.NaN
todaysFollowers[date] = np.NaN

In [8]:
pb = progress_bar(len(accounts_df))
print('Collecting Todays Tweets')
todaysTweets[date] = todaysTweets['Account'].apply(get_TodaysTweets)

Percent: [##########] 100.0% 

In [9]:
print('Collecting Todays Followers')
todaysFollowers[date] = todaysFollowers['Account'].apply(get_NumberofFollowers)

Earth_Pics[{'code': 17, 'message': 'No user matches for specified terms.'}]
fabulousanimals[{'code': 17, 'message': 'No user matches for specified terms.'}]


### Check

In [10]:
todaysFollowers.head()

,Account,2018-04-17
0,bonappetit,3267164.0
1,NatGeoFood,36174.0
2,TwitterFood,839511.0
3,Fooddotcom,31.0
4,newfoodeconomy,6508.0


In [11]:
todaysFollowers[todaysFollowers[date].isnull()] #check nulls

,Account,2018-04-17
49,Earth_Pics,NaN
146,fabulousanimals,NaN


In [12]:
todaysTweets.head()

,Account,2018-04-17
0,bonappetit,[{'created_at': 'Wed Apr 18 02:14:05 +0000 201...
1,NatGeoFood,[]
2,TwitterFood,[]
3,Fooddotcom,[]
4,newfoodeconomy,[{'created_at': 'Wed Apr 18 02:03:00 +0000 201...


In [13]:
todaysTweets[todaysTweets[date].isnull()] #check nulls

,Account,2018-04-17
49,Earth_Pics,NaN
146,fabulousanimals,NaN


# Save Data

In [ ]:
todaysFollowers.to_csv('Data/todaysFollowers_'+date+'.csv',sep=';',index=False)

In [ ]:
todaysTweets_json = todaysTweets.to_json(orient='records')
with open('Data/todaysTweets_'+date+'.json','w') as fp:
    json.dump(todaysTweets_json,fp)

# Add to Historic Data

### Load Historics

In [14]:
historicFollowers = pd.read_csv('Data/todaysFollowers_all.csv',sep = ';') #load historic Followers

### Concatenate

In [16]:
historicFollowers = pd.concat([historicFollowers,todaysFollowers[date]],axis=1) #add todays column

### Check

In [17]:
historicFollowers.head()

,Account,2018-02-11,2018-02-12,2018-02-13,2018-02-14,2018-02-15,2018-02-16,2018-02-17,2018-02-18,2018-02-19,2018-02-20,2018-04-17
0,bonappetit,3187723,3187464,3187274,3187123,3185897,3185752,3184465,3185259,3186173,3186218,3267164.0
1,NatGeoFood,35330,35345,35358,35365,35370,35387,35389,35398,35410,35414,36174.0
2,TwitterFood,838066,838115,838170,838213,838226,838229,838269,838229,838159,838130,839511.0
3,Fooddotcom,27,27,28,28,29,29,29,29,29,29,31.0
4,newfoodeconomy,5806,5815,5829,5829,5832,5845,5854,5866,5862,5875,6508.0


### Save

In [ ]:
historicFollowers.to_csv('Data/todaysFollowers_all.csv',sep=';',index=False) #save